In [50]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split

In [51]:
df=pd.read_csv(r'C:\Users\vedan\Machine Learning\Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [52]:
df['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [53]:
df.drop(columns=['RowNumber','Surname','CustomerId','HasCrCard','Tenure'],axis=1,inplace=True)

In [54]:
df=pd.get_dummies(df,columns=['Gender','Geography'])
list=['Gender_Female','Gender_Male','Geography_France','Geography_Germany','Geography_Spain']
for i in list:
    df[i]=df[i].astype(int)
df.head(10)

,CreditScore,Age,Balance,NumOfProducts,IsActiveMember,EstimatedSalary,Exited,Gender_Female,Gender_Male,Geography_France,Geography_Germany,Geography_Spain
0,619,42,0.00,1,1,101348.88,1,1,0,1,0,0
1,608,41,83807.86,1,1,112542.58,0,1,0,0,0,1
2,502,42,159660.80,3,0,113931.57,1,1,0,1,0,0
3,699,39,0.00,2,0,93826.63,0,1,0,1,0,0
4,850,43,125510.82,1,1,79084.10,0,1,0,0,0,1
5,645,44,113755.78,2,0,149756.71,1,0,1,0,0,1
6,822,50,0.00,2,1,10062.80,0,0,1,1,0,0
7,376,29,115046.74,4,0,119346.88,1,1,0,0,1,0
8,501,44,142051.07,2,1,74940.50,0,0,1,1,0,0
9,684,27,134603.88,1,1,71725.73,0,0,1,1,0,0


In [55]:
data=df

In [56]:
X=data.drop(columns=['Exited'],axis=1)
y=data['Exited']

In [66]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input

from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
X_scaled=scaler.fit_transform(X)

In [67]:
X_train,X_test,y_train,y_test=train_test_split(X_scaled,y,test_size=0.2,random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8000, 11)
(2000, 11)
(8000,)
(2000,)


In [68]:
tf.random.set_seed(1234)
model=Sequential([
    Input(shape=(11,)),
    Dense(128,activation='relu'),
    Dense(64,activation='relu'),
    Dense(32,activation='relu'),
    Dense(16,activation='relu'),
    Dense(1,activation='sigmoid')
])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                 │ (None, 128)            │         1,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,417 (48.50 KB)

 Trainable params: 12,417 (48.50 KB)

 Non-trainable params: 0 (0.00 B)

In [69]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [70]:
callback=tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [71]:
history=model.fit(X_train,y_train,validation_split=0.1,epochs=100,batch_size=256,verbose=1,callbacks=[callback])

Epoch 1/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7226 - loss: 0.5675 - val_accuracy: 0.8037 - val_loss: 0.4347
Epoch 2/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8075 - loss: 0.4437 - val_accuracy: 0.8263 - val_loss: 0.3995
Epoch 3/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8232 - loss: 0.4195 - val_accuracy: 0.8363 - val_loss: 0.3841
Epoch 4/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8363 - loss: 0.3974 - val_accuracy: 0.8475 - val_loss: 0.3671
Epoch 5/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8481 - loss: 0.3718 - val_accuracy: 0.8662 - val_loss: 0.3475
Epoch 6/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8576 - loss: 0.3513 - val_accuracy: 0.8575 - val_loss: 0.3397
Epoch 7/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8590 - loss: 0.3418 - val_accuracy: 0.8562 - val_loss: 0.3362
Epoch 8/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8624 - loss: 0.3371 - val_accuracy: 0.8537 - v

In [72]:
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
print(f"Train Accuracy: {train_acc:.4f}")

Train Accuracy: 0.8661


In [73]:
model.evaluate(X_test,y_test,verbose=1)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 876us/step - accuracy: 0.8625 - loss: 0.3406


[0.3406055271625519, 0.862500011920929]